# Modèle de Prédiction des Revenus de Films
## Clustering + Random Forest avec Optimisation Automatique
**Auteurs:** Martín Mondelli et González García Francisco
**Date:** 03/11/2025

## 1. Importation des Librairies

In [37]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, mean_squared_log_error
from sklearn.model_selection import train_test_split

# Scipy
from scipy.stats.mstats import winsorize

print("Librairies importées avec succès")

Librairies importées avec succès


## 2. Fonctions de Préprocessing

In [39]:
def replace_na(df, col):
    #Remplace les valeurs nulles et négatives par la médiane de la colonne
    median_value = df[col].median()
    df[col] = df[col].clip(lower=0)
    df[col] = df[col].fillna(0)        
    df[col] = df[col].replace(0, median_value)    
    return df[col]

def compute_outliers(df, col):
    #Calcule les limites pour les outliers en utilisant la méthode IQR
    Q1, Q3 = df[col].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    return lower, upper

def winsorize_df(df, col):
    #Applique la winsorisation pour limiter les outliers
    lim_inf, lim_sup = compute_outliers(df, col)
    series_w = df[col].copy()
    series_w = series_w.clip(lower=lim_inf, upper=lim_sup)    
    return series_w

print("Fonctions de préprocessing définies")

Fonctions de préprocessing définies


In [41]:
def rescaling_features(df):
#Crée des transformations et caractéristiques supplémentaires
    
    df = df.copy()
    
    # Transformations logarithmiques
    df['log_popularity_score'] = np.log(2 + df['popularity_score'])
    df['log_budget'] = np.log1p(df['budget'])
    
    # Transformations polynomiales
    df['length_2'] = df['length'] ** 2
    df['length_3'] = (2 + df['length']) ** 3
    df['popularity_score_2'] = df['popularity_score'] ** 2
    df['popularity_score_3'] = (2 + df['popularity_score']) ** 3
    df['budget_2'] = df['budget'] ** 2
    df['budget_3'] = (2 + df['budget']) ** 3
    df['budget_4'] = (df['budget']) ** 4
    
    return df

print("Fonction d'ingénierie des caractéristiques définie")

Fonction d'ingénierie des caractéristiques définie


In [43]:
def clean_dummies(df):
    
    #Crée des variables dummy et extrait les caractéristiques de date
    
    x_dum = df.copy()
    
    # Extraction de la date
    date_split = df["date"].str.split("/", expand=True)
    x_dum["month"] = date_split[0].astype(int)
    x_dum["day"] = date_split[1].astype(int)
    x_dum["year"] = date_split[2].astype(int)
    x_dum["year"] = np.where(x_dum["year"] > 27, 1900 + x_dum["year"], 2000 + x_dum["year"])
    
    # Variables dummy
    x_dum["sequels"] = df["collection"].apply(lambda c: 0 if pd.isna(c) else 1)
    x_dum["marketing"] = df["webpage"].apply(lambda c: 0 if pd.isna(c) else 1)
    x_dum["english"] = df["language"].apply(lambda c: 0 if c=="en" else 1)
    
    # Grandes compagnies (Top 10)
    big_companies = [
        "Twentieth Century Fox Film Corporation", "Universal Pictures", "Warner Bros. Pictures",
        "Columbia Pictures Corporation", "Walt Disney Pictures", "Marvel Studios", 
        "Paramount Pictures", "Legendary Pictures", "New Line Cinema", "DreamWorks Animation"
    ]
    x_dum["big_comp"] = df["company"].apply(
        lambda company: 1 if isinstance(company, str) and any(
            big_comp in company for big_comp in big_companies
        ) else 0
    )
    
    return x_dum

print("Fonction de création de variables dummy définie")

Fonction de création de variables dummy définie


## 3. Fonctions de Mise à l'Échelle et Clustering

In [45]:
def scale_features(train, test, cols):
    #Met à l'échelle les caractéristiques en utilisant StandardScaler
    train_scaled = train.copy()
    test_scaled = test.copy()

    for col in cols:
        scaler = StandardScaler()
        # Fit en train, transform en ambos
        train_scaled[col] = scaler.fit_transform(train[[col]])
        test_scaled[col] = scaler.transform(test[[col]])

    return train_scaled, test_scaled

print("Fonction de mise à l'échelle définie")

Fonction de mise à l'échelle définie


## 4. Fonction Principale du Modèle

In [47]:
def preprocess_data(df_train, df_test, target=None):
    
    #Pipeline complet de préprocessing des données
    # 1. Traitement des valeurs manquantes et outliers
    for col in ["budget", "length", "popularity_score"]:
        df_train[col] = replace_na(df_train, col)
        df_train[col] = winsorize_df(df_train, col)
        df_test[col] = replace_na(df_test, col)
        df_test[col] = winsorize_df(df_test, col)
    
    # 2. Transformations des caractéristiques
    df_train = rescaling_features(df_train)
    df_test = rescaling_features(df_test)
    
    # 3. Création de variables dummy
    df_train = clean_dummies(df_train)
    df_test = clean_dummies(df_test)
    
    # 4. Sélection des caractéristiques finales
    features = ["sequels", "big_comp", "budget", "budget_3", "length",
                "log_popularity_score", "year", "month"]
    
    df_train_final = df_train[features]
    df_test_final = df_test[features]
    
    print(f"Préprocessing terminé. Caractéristiques finales: {len(features)}")
    print(f"Caractéristiques: {features}")
    
    return df_train_final, df_test_final

print("Fonction de préprocessing définie")

Fonction de préprocessing définie


In [49]:
def train_clustered_model(X_train, y_train, X_test):
    #Entraîne le modèle complet: Clustering avec 3 clusters + Random Forest par cluster
    
    print("=== DÉBUT DE L'ENTRAÎNEMENT ===")
    
    # 1. Préparer les données pour le clustering
    num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    cols_to_scale = [c for c in num_cols if c not in ['year', 'month']]
    
    print(f"Colonnes à mettre à l'échelle pour clustering: {cols_to_scale}")
    
    # 2. Mise à l'échelle
    X_train_scaled, X_test_scaled = scale_features(X_train, X_test, cols_to_scale)
    
    # 3. Clustering avec 3 clusters (fixe)
    n_clusters = 3
    print(f"Utilisation de {n_clusters} clusters pour le modèle")
    
    # 4. Appliquer le clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    train_clusters = kmeans.fit_predict(X_train_scaled[num_cols])
    test_clusters = kmeans.predict(X_test_scaled[num_cols])
    
    # Informations sur les clusters
    cluster_sizes = pd.Series(train_clusters).value_counts().sort_index()
    print(f"\nTailles des clusters dans l'entraînement:")
    for i, size in cluster_sizes.items():
        print(f"  Cluster {i}: {size} observations ({size/len(train_clusters)*100:.1f}%)")
    
    # 5. Entraîner Random Forest par cluster
    print("\nEntraînement des Random Forest par cluster...")
    cluster_models = {}
    
    for cluster in range(n_clusters):
        print(f"  Entraînement cluster {cluster}...")
        
        # Sélectionner les données du cluster
        mask = train_clusters == cluster
        X_cluster = X_train_scaled.iloc[mask]
        y_cluster = np.log1p(y_train.iloc[mask].values.ravel())
        
        # Entraîner le modèle
        model = RandomForestRegressor(
            n_estimators=400,
            max_depth=15,
            min_samples_leaf=2,
            max_features='sqrt',
            random_state=0,
            n_jobs=-1
        )
        model.fit(X_cluster[num_cols], y_cluster)
        cluster_models[cluster] = model
    
    # 6. Faire les prédictions
    y_pred_log = np.zeros(X_test_scaled.shape[0])
    
    for cluster in range(n_clusters):
        mask = test_clusters == cluster
        if np.any(mask):
            X_cluster_test = X_test_scaled.iloc[mask]
            y_pred_log[mask] = cluster_models[cluster].predict(X_cluster_test[num_cols])
    
    # 7. Inverser la transformation logarithmique
    y_pred = np.expm1(y_pred_log)
    
    # Informations du modèle
    cluster_info = {
        'n_clusters': n_clusters,
        'cluster_sizes': cluster_sizes.to_dict(),
        'test_cluster_distribution': pd.Series(test_clusters).value_counts().sort_index().to_dict()
    }
    return y_pred, cluster_info


## 5. Fonction pour Formater les Résultats

In [51]:
def format_predictions(y_pred):
    #Formate les prédictions comme string séparé par des virgules
    str_results = [str(int(np.round(pred, 0))) for pred in y_pred]
    return ', '.join(str_results)

print("Fonction de formatage définie")

Fonction de formatage définie


---
## 6. SECTION D'ÉVALUATION
**Cette section sera exécutée par le professeur avec les données privées**

### Chargement des Données
**INSTRUCTIONS POUR LE PROFESSEUR:**
- Remplacer les URLs par les chemins des fichiers de données privées
- Conserver les noms de variables: `df_train_in`, `y_train`, `df_test`

In [53]:
# DONNÉES D'ENTRAÎNEMENT ET DE TEST
# TODO: Remplacer ces URLs par ses fichiers privés

df_train_in = pd.read_csv("https://edouardpauwels.fr/MLM2DSSS/challenge_train_features.csv", index_col=0)
y_train = pd.read_csv("https://edouardpauwels.fr/MLM2DSSS/challenge_train_revenue.csv", index_col=0)
df_test = pd.read_csv("https://edouardpauwels.fr/MLM2DSSS/challenge_test_features.csv", index_col=0)
display(df_train_in.head())

,title,language,country,length,date,genre,summary,collection,company,webpage,poster_file,cast,keywords,crew,popularity_score,budget
0,Rocky V,en,US,104.0,10/18/90,Drama,A lifetime of taking shots has ended Rocky's c...,Rocky Collection,United Artists,NaN,G6shAlq8wwud6byWm13tLuiR2P5.jpg,"Sylvester Stallone,Talia Shire,Burt Young,Sage...","philadelphia,transporter,father son relationsh...","[{'job': 'Director', 'name': 'John G. Avildsen...",14.007329,42000000
1,Oculus,en,US,104.0,9/8/13,Horror,A woman tries to exonerate her brother's murde...,NaN,"Intrepid Pictures,Blumhouse Productions,Relati...",NaN,HcPqD6LzKOPqXkLocfSly49C8ci.jpg,"Karen Gillan,Brenton Thwaites,Katee Sackhoff,J...","hallucination,supernatural,mirror,skepticism,g...","[{'job': 'Casting', 'name': 'Anne McCarthy'}, ...",8.698043,5000000
2,Tomorrowland,en,US,130.0,5/19/15,"Adventure,Family,Mystery,Science Fiction","Bound by a shared destiny, a bright, optimisti...",NaN,"Walt Disney Pictures,Babieka,A113",http://movies.disney.com/tomorrowland,MTBS6htgG0g2EUf93yZQNV9zC96.jpg,"Britt Robertson,George Clooney,Raffey Cassidy,...","inventor,apocalypse,destiny,imax,dreamer,futur...","[{'job': 'Editor', 'name': 'Craig Wood'}, {'jo...",22.296076,190000000
3,Things Are Tough All Over,en,US,90.0,8/4/82,"Action,Comedy",Cheech and Chong are hired to drive a limo fro...,Cheech & Chong Collection,C & C Brown Production,NaN,vxdj0agTm8reLtxXuzQVkYYx7oA.jpg,"Cheech Marin,Tommy Chong,Toni Attell,Mike Baca...","pornography,chicago,money laundering,gas stati...","[{'job': 'Writer', 'name': 'Cheech Marin'}, {'...",9.442756,0
4,How to Be Single,en,US,110.0,1/21/16,"Comedy,Romance",New York City is full of lonely hearts seeking...,NaN,"New Line Cinema,Flower Films,Metro-Goldwyn-May...",http://howtobesinglemovie.com/,O8UMYar5eX8iYY5c8qTKO4rFhAj.jpg,"Dakota Johnson,Rebel Wilson,Alison Brie,Nichol...","new york,based on novel,one-night stand,single","[{'job': 'Casting', 'name': 'Avy Kaufman'}, {'...",8.898988,38000000


### Préprocessing des Données

In [55]:
# Appliquer le préprocessing complet
X_train_processed, X_test_processed = preprocess_data(df_train_in.copy(), df_test.copy(), y_train)

Préprocessing terminé. Caractéristiques finales: 8
Caractéristiques: ['sequels', 'big_comp', 'budget', 'budget_3', 'length', 'log_popularity_score', 'year', 'month']


### Entraînement du Modèle

In [57]:
# Entraîner le modèle avec 3 clusters
predictions, model_info = train_clustered_model(
    X_train_processed, 
    y_train, 
    X_test_processed
)

=== DÉBUT DE L'ENTRAÎNEMENT ===
Colonnes à mettre à l'échelle pour clustering: ['sequels', 'big_comp', 'budget', 'budget_3', 'length', 'log_popularity_score']
Utilisation de 3 clusters pour le modèle

Tailles des clusters dans l'entraînement:
  Cluster 0: 682 observations (34.1%)
  Cluster 1: 118 observations (5.9%)
  Cluster 2: 1200 observations (60.0%)

Entraînement des Random Forest par cluster...
  Entraînement cluster 0...
  Entraînement cluster 1...
  Entraînement cluster 2...


### Résultats Finaux

In [59]:
# Formater les prédictions pour la livraison
result_string = format_predictions(predictions)

print("=== PRÉDICTIONS FINALES ===")
print(result_string)
print(f"\n=== RÉSUMÉ ===")
print(f"Total des prédictions: {len(predictions)}")
print(f"Modèle utilisé: Clustering ({model_info['n_clusters']} clusters) + Random Forest")
print(f"Informations du modèle final:")
print(f"  - Nombre de clusters: {model_info['n_clusters']}")
print(f"  - Distribution des clusters dans l'entraînement: {model_info['cluster_sizes']}")
print(f"  - Distribution des clusters dans le test: {model_info['test_cluster_distribution']}")


=== PRÉDICTIONS FINALES ===
96704075, 6193138, 2656040, 8882629, 1550422, 108381944, 18546605, 38221536, 2515100, 58930139, 57160921, 976972, 40267465, 8108776, 1158704, 8777461, 2459817, 3058598, 3020207, 7257316, 15536941, 57440753, 13574541, 102212, 11218523, 637465, 2132448, 4783198, 442717, 63205, 62556193, 26407233, 5389237, 28549499, 2398986, 114143952, 5984074, 2672016, 20816191, 799193, 10673539, 26071645, 32220631, 175478514, 44305963, 6548915, 3036525, 1088616, 14041889, 28601306, 165303532, 28448912, 332034443, 7363610, 5808092, 19210413, 759891, 58325931, 196470846, 6023609, 5063543, 4211587, 8556088, 53530374, 12180098, 1120661, 1388701, 2270729, 55857017, 228539622, 17317251, 2599778, 54285, 65369, 4318357, 347299, 62688534, 1606372, 14040270, 17472618, 1365191, 10822786, 54863721, 878876, 4221861, 17718852, 23503794, 42289609, 1891685, 53383887, 58311511, 73262614, 5249666, 7847649, 23981600, 2470633, 1514734, 702575, 21798586, 30511616, 256561376, 4120993, 20441310, 10